In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
from Korpora import Korpora

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

사용 디바이스: cuda:0


In [5]:
import pandas as pd

train = pd.read_csv('/home/kdt-admin/lsy/preprocess2.csv')
train=train[['clean1','malicious']]
test = pd.read_csv('/home/kdt-admin/lsy/preprocess2_test.csv')
train.sample(6)

,clean1,malicious
18284,형님 안녕하세요 정말로 형님의 여동생분과 교제를 하고 싶습니다 꼭 연락주세요,0
2364,그냥 돈 더보태서 보잉 사자,0
7404,둘이 포옹 한가인이 싫어할까 백종원이 싫어할까,1
4581,역시 렉서스 배운자의차,0
17906,양한승목사 앵벌이 이래서 교회가 미안합니다,1
22184,솔직하게 글써라 좆 빨렸지,1


In [6]:
train['length'] = train['clean1'].apply(lambda x: len(str(x)))
test['length'] = test['clean1'].apply(lambda x: len(str(x)))

In [7]:
train = train.loc[train['length'] >= 5]
test = test.loc[test['length'] >= 5]
train

,clean1,malicious,length
0,석유 화학은 오일 현대오일뱅크지,0,18
1,은교입니다,0,6
2,김트루는 디씨네임드야,0,12
3,노노 그냥 해논 닉이도 구로디지탈산다 키 잘생긴편인데 머하노 인물아깝게 인생짧아,0,45
4,존경 스롭넹 와우,0,10
...,...,...,...
28063,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐,1,33
28064,갱상도가아니라 홍어지 개좃같은 홍어년들,1,22
28065,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,25
28066,링크도 안박고 가서 글 쓰자고 선동하네,1,22


In [ ]:
train = train.loc[train['length'] >= 5]
test = test.loc[test['length'] >= 5]
train

,clean1,malicious,length
0,석유 화학은 오일 현대오일뱅크지,0,18
1,은교입니다,0,6
2,김트루는 디씨네임드야,0,12
3,노노 그냥 해논 닉이도 구로디지탈산다 키 잘생긴편인데 머하노 인물아깝게 인생짧아,0,45
4,존경 스롭넹 와우,0,10
...,...,...,...
28063,성소수자 이빈 이띤 좆같고 뱡신 같은 말은 누가 만드는거냐,1,33
28064,갱상도가아니라 홍어지 개좃같은 홍어년들,1,22
28065,말레이시아랑 인도네시아 여자 존나 못생겼던데,1,25
28066,링크도 안박고 가서 글 쓰자고 선동하네,1,22


In [8]:
CHECKPOINT_NAME = 'kykim/bert-kor-base'

In [9]:
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader

In [10]:
class TokenDataset(Dataset):
  
    def __init__(self, dataframe, tokenizer_pretrained):
        # sentence, label 컬럼으로 구성된 데이터프레임 전달
        self.data = dataframe        
        # Huggingface 토크나이저 생성
        self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, idx):
        sentence = self.data.iloc[idx]['clean1']
        label = self.data.iloc[idx]['malicious']

        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )

        input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
        attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
        token_type_ids = torch.zeros_like(attention_mask)

        # input_ids, attention_mask, token_type_ids 이렇게 3가지 요소를 반환하도록 합니다.
        # input_ids: 토큰
        # attention_mask: 실제 단어가 존재하면 1, 패딩이면 0 (패딩은 0이 아닐 수 있습니다)
        # token_type_ids: 문장을 구분하는 id. 단일 문장인 경우에는 전부 0
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask, 
            'token_type_ids': token_type_ids,
        }, torch.tensor(label)

In [11]:
# 토크나이저 지정
tokenizer_pretrained = CHECKPOINT_NAME

# train, test 데이터셋 생성
train_data = TokenDataset(train, tokenizer_pretrained)
test_data = TokenDataset(test, tokenizer_pretrained)

# DataLoader로 이전에 생성한 Dataset를 지정하여, batch 구성, shuffle, num_workers 등을 설정합니다.
train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

/home/kdt-admin/miniconda3/envs/leesy/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [12]:
# 1개의 batch 꺼내기
inputs, labels = next(iter(train_loader))

# 데이터셋을 device 설정
inputs = {k: v.to(device) for k, v in inputs.items()}
labels.to(device)

tensor([1, 1, 1, 1, 1, 1, 0, 1], device='cuda:0')

In [13]:
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids'])

In [14]:
# key 별 shape 확인
inputs['input_ids'].shape, inputs['attention_mask'].shape, inputs['token_type_ids'].shape

(torch.Size([8, 512]), torch.Size([8, 512]), torch.Size([8, 512]))

In [15]:
from transformers import BertConfig

config = BertConfig.from_pretrained(CHECKPOINT_NAME)
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "embedding_size": 768,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 42000
}

In [16]:
from transformers import BertModel

# 모델 생성
model_bert = BertModel.from_pretrained(CHECKPOINT_NAME).to(device)
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(42000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [17]:
# 이전에 생성한 inputs를 한 번 대입해본 후 결과를 확인
output = model_bert(**inputs)
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [18]:
# last_hidden_state 출력
last_hidden_state = output['last_hidden_state']
print(last_hidden_state.shape)
print(last_hidden_state[:, 0, :])

torch.Size([8, 512, 768])
tensor([[ 0.1672,  0.0050,  0.7143,  ...,  0.1032, -1.5025,  1.0528],
        [ 0.1925, -0.4371, -0.2416,  ...,  0.2397, -0.7254,  0.1815],
        [ 0.1959, -0.2967,  0.4299,  ...,  0.3868, -0.3868,  0.2145],
        ...,
        [-0.0784, -0.2149,  0.8533,  ..., -0.3312, -0.0875, -0.0956],
        [-0.7371,  0.5831, -0.1502,  ...,  0.5480, -0.5987,  0.5707],
        [-0.9204, -0.5608, -0.1486,  ..., -0.6979, -0.0264,  0.4394]],
       device='cuda:0', grad_fn=<SliceBackward0>)


In [19]:
# pooler_output 출력
pooler_output = output['pooler_output']
print(pooler_output.shape)
print(pooler_output)

torch.Size([8, 768])
tensor([[-0.8237,  0.4360, -0.4181,  ..., -0.8549,  0.2616,  0.7425],
        [ 0.0532,  0.2563, -0.9547,  ..., -0.7319,  0.2855,  0.8145],
        [-0.6320,  0.0056, -0.8062,  ..., -0.5025,  0.1880,  0.7717],
        ...,
        [-0.8292,  0.3388, -0.9223,  ...,  0.0147,  0.0442,  0.6598],
        [ 0.9795, -0.1802, -0.9994,  ..., -0.7313,  0.7364,  0.7873],
        [ 0.8975, -0.6296, -0.9999,  ..., -0.8566,  0.8213,  0.6538]],
       device='cuda:0', grad_fn=<TanhBackward0>)


In [20]:
fc = nn.Linear(768, 2)
fc.to(device)
fc_output = fc(last_hidden_state[:, 0, :])
print(fc_output.shape)
print(fc_output.argmax(dim=1))

torch.Size([8, 2])
tensor([0, 0, 0, 0, 0, 1, 1, 1], device='cuda:0')


사전학습 bert모델 fine-tuning

In [21]:
class CustomBertModel(nn.Module):
    def __init__(self, bert_pretrained, dropout_rate=0.5):
        # 부모클래스 초기화
        super(CustomBertModel, self).__init__()
        # 사전학습 모델 지정
        self.bert = BertModel.from_pretrained(bert_pretrained)
        # dropout 설정
        self.dr = nn.Dropout(p=dropout_rate)
        # 최종 출력층 정의
        self.fc = nn.Linear(768, 2)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # 입력을 pre-trained bert model 로 대입
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # 결과의 last_hidden_state 가져옴
        last_hidden_state = output['last_hidden_state']
        # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
        x = self.dr(last_hidden_state[:, 0, :])
        # FC 을 거쳐 최종 출력
        x = self.fc(x)
        return x

In [22]:
bert = CustomBertModel(CHECKPOINT_NAME)
bert.to(device)

CustomBertModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [23]:
# loss 정의: CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()

# 옵티마이저 정의: bert.paramters()와 learning_rate 설정
optimizer = optim.Adam(bert.parameters(), lr=1e-5)

In [24]:
from tqdm import tqdm  # Progress Bar 출력

def model_train(model, data_loader, loss_fn, optimizer, device):
    # 모델을 훈련모드로 설정합니다. training mode 일 때 Gradient 가 업데이트 됩니다. 반드시 train()으로 모드 변경을 해야 합니다.
    model.train()
    
    # loss와 accuracy 계산을 위한 임시 변수 입니다. 0으로 초기화합니다.
    running_loss = 0
    corr = 0
    counts = 0
    
    # 예쁘게 Progress Bar를 출력하면서 훈련 상태를 모니터링 하기 위하여 tqdm으로 래핑합니다.
    prograss_bar = tqdm(data_loader, unit='batch', total=len(data_loader), mininterval=1)
    
    # mini-batch 학습을 시작합니다.
    for idx, (inputs, labels) in enumerate(prograss_bar):
        # inputs, label 데이터를 device 에 올립니다. (cuda:0 혹은 cpu)
        inputs = {k:v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)
        
        # 누적 Gradient를 초기화 합니다.
        optimizer.zero_grad()
        
        # Forward Propagation을 진행하여 결과를 얻습니다.
        output = model(**inputs)
        
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, labels)
        
        # 오차역전파(Back Propagation)을 진행하여 미분 값을 계산합니다.
        loss.backward()
        
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()
        
        # output의 max(dim=1)은 max probability와 max index를 반환합니다.
        # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
        _, pred = output.max(dim=1)
        
        # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
        # 합계는 corr 변수에 누적합니다.
        corr += pred.eq(labels).sum().item()
        counts += len(labels)
        
        # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
        # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
        # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
        running_loss += loss.item() * labels.size(0)
        
        # 프로그레스바에 학습 상황 업데이트
        prograss_bar.set_description(f"training loss: {running_loss/(idx+1):.5f}, training accuracy: {corr / counts:.5f}")
        
    # 누적된 정답수를 전체 개수로 나누어 주면 정확도가 산출됩니다.
    acc = corr / len(data_loader.dataset)
    
    # 평균 손실(loss)과 정확도를 반환합니다.
    # train_loss, train_acc
    return running_loss / len(data_loader.dataset), acc

In [25]:
def model_evaluate(model, data_loader, loss_fn, device):
    model.eval()
    
    # Gradient가 업데이트 되는 것을 방지 
    with torch.no_grad():
        corr = 0
        running_loss = 0
        
        for inputs, labels in data_loader:
            inputs = {k:v.to(device) for k, v in inputs.items()}
            labels = labels.to(device)
            
            # 모델에 Forward Propagation을 하여 결과를 도출합니다.
            output = model(**inputs)
            
            # output의 max(dim=1)은 max probability와 max index를 반환합니다.
            # max probability는 무시하고, max index는 pred에 저장하여 label 값과 대조하여 정확도를 도출합니다.
            _, pred = output.max(dim=1)
            
            # pred.eq(lbl).sum() 은 정확히 맞춘 label의 합계를 계산합니다. item()은 tensor에서 값을 추출합니다.
            # 합계는 corr 변수에 누적합니다.
            corr += torch.sum(pred.eq(labels)).item()
            
            # loss 값은 1개 배치의 평균 손실(loss) 입니다. img.size(0)은 배치사이즈(batch size) 입니다.
            # loss 와 img.size(0)를 곱하면 1개 배치의 전체 loss가 계산됩니다.
            # 이를 누적한 뒤 Epoch 종료시 전체 데이터셋의 개수로 나누어 평균 loss를 산출합니다.
            running_loss += loss_fn(output, labels).item() * labels.size(0)
        
        # validation 정확도를 계산합니다.
        # 누적한 정답숫자를 전체 데이터셋의 숫자로 나누어 최종 accuracy를 산출합니다.
        acc = corr / len(data_loader.dataset)
        
        # 결과를 반환합니다.
        # val_loss, val_acc
        return running_loss / len(data_loader.dataset), acc

In [26]:
num_epochs = 10
model_name = 'bert-kor-base'

min_loss = np.inf

for epoch in range(num_epochs):
    # Model Training
    train_loss, train_acc = model_train(bert, train_loader, loss_fn, optimizer, device)

    # 검증 손실과 검증 정확도를 반환 받습니다.
    val_loss, val_acc = model_evaluate(bert, test_loader, loss_fn, device)   
    
    # val_loss 가 개선되었다면 min_loss를 갱신하고 model의 가중치(weights)를 저장합니다.
    if val_loss < min_loss:
        print(f'[INFO] val_loss has been improved from {min_loss:.5f} to {val_loss:.5f}. Saving Model!')
        min_loss = val_loss
        torch.save(bert.state_dict(), f'{model_name}_{epoch}.pth')
    
    # Epoch 별 결과를 출력합니다.
    print(f'epoch {epoch+1:02d}, loss: {train_loss:.5f}, acc: {train_acc:.5f}, val_loss: {val_loss:.5f}, val_accuracy: {val_acc:.5f}')

  0%|          | 0/3468 [00:00<?, ?batch/s]/home/kdt-admin/miniconda3/envs/leesy/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 6, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
training loss: 3.62671, training accuracy: 0.78453: 100%|██████████| 3468/3468 [1:14:49<00:00,  1.29s/batch]


[INFO] val_loss has been improved from inf to 2.64483. Saving Model!
epoch 01, loss: 0.45340, acc: 0.78453, val_loss: 2.64483, val_accuracy: 0.27630


training loss: 2.68005, training accuracy: 0.85306: 100%|██████████| 3468/3468 [1:14:51<00:00,  1.30s/batch]


epoch 02, loss: 0.33505, acc: 0.85306, val_loss: 2.89413, val_accuracy: 0.27693


training loss: 1.66412, training accuracy: 0.91732:   9%|▉         | 319/3468 [06:55<1:08:23,  1.30s/batch]


KeyboardInterrupt: 

In [ ]:
bert.load_state_dict(torch.load(f'{model_name}.pth'))

In [ ]:
class CustomPredictor():
    def __init__(self, model, tokenizer, labels: dict):
        self.model = model
        self.tokenizer = tokenizer
        self.labels = labels
        
    def predict(self, sentence):
        # 토큰화 처리
        tokens = self.tokenizer(
            sentence,                # 1개 문장 
            return_tensors='pt',     # 텐서로 반환
            truncation=True,         # 잘라내기 적용
            padding='max_length',    # 패딩 적용
            add_special_tokens=True  # 스페셜 토큰 적용
        )
        tokens.to(device)
        prediction = self.model(**tokens)
        prediction = F.softmax(prediction, dim=1)
        output = prediction.argmax(dim=1).item()
        prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
        print(f'[{result}]\n확률은: {prob*100:.3f}% 입니다.')